In [1]:
import pyspark 
from pyspark import SparkContext, SparkConf

from pyspark.sql import SparkSession

sc = SparkContext(conf = SparkConf())
spark = SparkSession.builder.appName("ass8").getOrCreate()


In [2]:
import os
import re
import random
import time
import binascii
from bisect import bisect_right
from heapq import heappop, heappush

import numpy as np
import pandas as pd

import lsh

In [3]:
from collections import defaultdict

def makeA(d):
    dd = defaultdict(int, d)
    return [dd[n] for n in range(len(d))]


In [4]:
#path = "data/"
#df = spark.read \
#    .option("header", False) \
#    .option("multiline", True) \
#    .option("inferSchema", True) \
#    .text('testdatahw1.txt')

In [5]:
numHashes = 10
numDocs = 18506

In [6]:
print("Shingling articles...")

# The current shingle ID value to assign to the next new shingle we 
# encounter. When a shingle gets added to the dictionary, we'll increment this
# value.
curShingleID = 0

# Create a dictionary of the articles, mapping the article identifier (e.g., 
# "t8470") to the list of shingle IDs that appear in the document.
docsAsShingleSets = {};
  
# Open the data file.
f = open('testdatahw1.txt', "r")

docNames = []

t0 = time.time()

totalShingles = 0

for i in range(0, numDocs):
  
  # Read all of the words (they are all on one line) and split them by white
  # space.
  words = f.readline().split(" ") 
  
  # Retrieve the article ID, which is the first word on the line.  
#  docID = words[0]
  docID = i
  
  # Maintain a list of all document IDs.  
  docNames.append(docID)
    
  del words[0]  
  
  # 'shinglesInDoc' will hold all of the unique shingle IDs present in the 
  # current document. If a shingle ID occurs multiple times in the document,
  # it will only appear once in the set (this is a property of Python sets).
  shinglesInDoc = set()
  
  # For each word in the document...
  for index in range(0, len(words) - 2):
    # Construct the shingle text by combining three words together.
    shingle = words[index] + " " + words[index + 1] + " " + words[index + 2]
    
    # Hash the shingle to a 32-bit integer.
    crc = binascii.crc32(shingle.encode()) & 0xffffffff
#    crc = binascii.crc32(shingle) & 0xffffffff
#    crc = binascii.a2b_base64(shingle + "=")
    
    # Add the hash value to the list of shingles for the current document. 
    # Note that set objects will only add the value to the set if the set 
    # doesn't already contain it. 
    shinglesInDoc.add(crc)
  
  # Store the completed list of shingles for this document in the dictionary.
  docsAsShingleSets[docID] = shinglesInDoc
  
  # Count the number of shingles across all documents.
  totalShingles = totalShingles + (len(words) - 2)

# Close the data file.  
f.close()  

# Report how long shingling took.
print( '\nShingling ' + str(numDocs) + ' docs took %.2f sec.' % (time.time() - t0))
 
print( '\nAverage shingles per doc: %.2f' % (totalShingles / numDocs))


Shingling articles...

Shingling 18506 docs took 3.06 sec.

Average shingles per doc: 100.18


In [7]:
df_done = makeA(docsAsShingleSets)

In [8]:
df_final = np.array(df_done)

In [9]:
df_test2 = pd.DataFrame(df_final, columns = ['values'])

In [10]:
ts = pd.Series(pd.arrays.SparseArray(df_test2["values"]))

In [11]:
fix = []

for row in ts:
   tmp = []

   if len(row) == 0:
       continue

   for item in row:
       tmp.append(item)
   fix.append(tmp)

data = sc.parallelize(fix)

In [12]:
m = 1000
n = 1000
b = 25
c = 2
p = 65537

In [13]:
model = lsh.run(data, p, m, n, b, c)

In [14]:
print('Found %s clusters.' % model.buckets.count())

Found 1349 clusters.
